In [ ]:
import requests
import mysql.connector

# API URL
url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=S0f2tGc7vYfWYFivhBHOC0GzJ0pcisRg96egUToU"

# Fetch data from API
response = requests.get(url)
if response.status_code != 200:
    print(f"Error fetching API data: {response.status_code}")
    exit()

data = response.json()  # Convert response to JSON


db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="doubles"
)
cursor = db.cursor()


create_table_query = """
CREATE TABLE IF NOT EXISTS Competitor_Rankings (
    rank_id INT PRIMARY KEY AUTO_INCREMENT,
    rank INT NOT NULL,
    movement INT NOT NULL,
    points INT NOT NULL,
    competitions_played INT NOT NULL,
    competitor_id VARCHAR(50),
    FOREIGN KEY (competitor_id) REFERENCES Competitors(competitor_id) ON DELETE CASCADE
);
"""
cursor.execute(create_table_query)
db.commit()


insert_query = """
INSERT INTO Competitor_Rankings (rank, movement, points, competitions_played, competitor_id)
VALUES (%s, %s, %s, %s, %s)
"""

for ranking in data.get('rankings', []):  # Ensure 'rankings' key exists
    for competitor_ranking in ranking.get('competitor_rankings', []):  # Ensure 'competitor_rankings' key exists
        competitor = competitor_ranking.get('competitor', {})  # Get competitor data safely
        values = (
            competitor_ranking.get('rank'),
            competitor_ranking.get('movement'),
            competitor_ranking.get('points'),
            competitor_ranking.get('competitions_played'),
            competitor.get('id')  # Competitor ID
        )
        try:
            cursor.execute(insert_query, values)
        except mysql.connector.Error as err:
            print(f"Error inserting {values}: {err}")

db.commit()
cursor.close()
db.close()

print("Data successfully inserted into Competitor_Rankings table!")
